<a href="https://colab.research.google.com/github/ponOlgap/OLga/blob/main/nlp_sentiment_simple_vectorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Определение тональности отзывов на банки с помощью классических алгоритмов машинного обучения 

Используем логистическую регрессию и мешок слов.



In [ ]:
!pip install pymorphy2[fast] 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 29.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 371.1/371.1 KB 40.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for DAWG: filename=DAWG-0.8.0-cp38-cp38-linux_x86_64.whl size=936218 sha256=048049d065c0410db4627874dc89d3d079ada25afa6e6f5ae092413710b311c2
  Stored in directory: /root/.cache/pip/wheels/1c/e6/8f/313a7ccc57b29a7affb7205664277a1d5ebe73bf600a69a615
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=c1189703f547fb3362333adf905284ae1ef4ad4bec58adde1a3ff170d9829840
  Stored in directory: /root/.cache/pip/wheels/56/ea/58/ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c
Successfully built DAWG docopt


In [ ]:
import pandas as pd
import numpy as np
import pymorphy2
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from collections import Counter

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Константы

In [ ]:
max_words = 10000
random_state = 42

## Загружаем и готовим набор данных

In [ ]:
!wget https://www.dropbox.com/s/a9r0b2yj3vqvi13/banks.csv?dl=1 -O banks.csv

--2023-01-04 12:59:48--  https://www.dropbox.com/s/a9r0b2yj3vqvi13/banks.csv?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.4.18, 2620:100:6019:18::a27d:412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.4.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/a9r0b2yj3vqvi13/banks.csv [following]
--2023-01-04 12:59:48--  https://www.dropbox.com/s/dl/a9r0b2yj3vqvi13/banks.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc9725cbaa9e1a68b371466a787c.dl.dropboxusercontent.com/cd/0/get/Bz4lrVksNuNGncN_do-sbrbcoQ5G_nzEGHz_kuQ930SwQ_VvhluvMjTi1fh89kpu9rgPUjJw9f5ySTyeezNdneDX6zk1znCjeY1_ayS8YcwvMoyCpaO-mBJneCo2P0vM5P38PAGiPmbaRoLYnEcCwyJLQL0VtnYn2vf01zOEjYOt5g/file?dl=1# [following]
--2023-01-04 12:59:48--  https://uc9725cbaa9e1a68b371466a787c.dl.dropboxusercontent.com/cd/0/get/Bz4lrVksNuNGncN_do-sbrbcoQ5G_nzEGHz_kuQ930SwQ_VvhluvMjTi1fh89kpu9rgPUjJw9f5ySTyeezNdneD

In [ ]:
banks = pd.read_csv('banks.csv', sep='\t', index_col='idx');

In [ ]:
def preprocess(text, stop_words, punctuation_marks, morph):
    tokens = word_tokenize(text.lower())
    preprocessed_text = []
    for token in tokens:
        if token not in punctuation_marks:
            lemma = morph.parse(token)[0].normal_form
            if lemma not in stop_words:
                preprocessed_text.append(lemma)
    return preprocessed_text

In [ ]:
punctuation_marks = ['!', ',', '(', ')', ':', '-', '?', '.', '..', '...', '«', '»', ';', '–', '--']
stop_words = stopwords.words("russian")
morph = pymorphy2.MorphAnalyzer()

In [ ]:
banks['Preprocessed_texts'] = banks.apply(lambda row: preprocess(row['Text'], punctuation_marks, stop_words, morph), axis=1)

Считаем частоту слов во всех отзывах

In [ ]:
words = Counter()

In [ ]:
for txt in banks['Preprocessed_texts']:
    words.update(txt)

Создаем словарь, упорядоченный по частоте

В словаре будем использовать 2 специальных кода:
- Код заполнитель: 0
- Неизвестное слово: 1

Нумерация слов в словаре начинается с 2.

In [ ]:
# Словарь, отображающий слова в коды
word_to_index = dict()
# Словарь, отображающий коды в слова
index_to_word = dict()

Создаем словари

In [ ]:
for i, word in enumerate(words.most_common(max_words - 2)):
    word_to_index[word[0]] = i + 2
    index_to_word[i + 2] = word[0]

Функция для преобразования списка слов в список кодов

In [ ]:
def text_to_sequence(txt, word_to_index):
    seq = []
    for word in txt:
        index = word_to_index.get(word, 1) # 1 означает неизвестное слово
        # Неизвестные слова не добавляем в выходную последовательность
        if index != 1:
            seq.append(index)
    return seq

Преобразуем все тексты в последовательность кодов слов

In [ ]:
banks['Sequences'] = banks.apply(lambda row: text_to_sequence(row['Preprocessed_texts'], word_to_index), axis=1)

In [ ]:
banks

,Score,Text,Preprocessed_texts,Sequences
idx,,,,
0,Positive,В Альфа-Банке работает замечательная девушка -...,"[альфа-банк, работать, замечательный, девушка,...","[62, 43, 916, 78, 204, 1867, 357, 2711, 409, 1..."
1,Negative,Оформляя рассрочку в м. Видео в меге тёплый ст...,"[оформлять, рассрочка, м., видео, мег, тёплый,...","[267, 855, 1178, 3215, 3892, 2982, 7347, 171, ..."
2,Positive,Очень порадовала оперативность работы в банке....,"[очень, порадовать, оперативность, работа, бан...","[18, 1055, 910, 29, 2, 475, 183, 3, 628, 1768,..."
3,Negative,Имела неосторожность оформить потреб. кредит в...,"[иметь, неосторожность, оформить, потреба, кре...","[118, 4818, 72, 2368, 9, 62, 18, 2665, 1287, 3..."
4,Negative,Небольшая предыстория: Нашел на сайте MDM банк...,"[небольшой, предыстория, найти, сайт, mdm, бан...","[418, 3851, 275, 87, 2, 648, 3, 4894, 2, 296, ..."
...,...,...,...,...
13994,Positive,"О высокой надёжности МКБ, порядочности и добро...","[высокий, надёжность, мкб, порядочность, добро...","[401, 2321, 809, 5028, 8159, 7, 3099, 126, 441..."
13995,Positive,"Обслуживаюсь в офисе на Чернореченской 42а, ка...","[обслуживаться, офис, чернореченский, 42а, физ...","[362, 24, 9494, 2152, 201, 14, 24, 96, 141, 66..."
13996,Positive,Попала сегодня в очень неприятную ситуацию. Ре...,"[попасть, сегодня, очень, неприятный, ситуация...","[517, 109, 18, 992, 53, 38, 224, 233, 3, 46, 3..."


## Готовим данные для обучения

Преобразуем текстовые метки классов в числовые

In [ ]:
mapping = {'Negative': 0, 'Positive': 1}

In [ ]:
banks.replace({'Score': mapping}, inplace=True)

In [ ]:
banks

,Score,Text,Preprocessed_texts,Sequences
idx,,,,
0,1,В Альфа-Банке работает замечательная девушка -...,"[альфа-банк, работать, замечательный, девушка,...","[62, 43, 916, 78, 204, 1867, 357, 2711, 409, 1..."
1,0,Оформляя рассрочку в м. Видео в меге тёплый ст...,"[оформлять, рассрочка, м., видео, мег, тёплый,...","[267, 855, 1178, 3215, 3892, 2982, 7347, 171, ..."
2,1,Очень порадовала оперативность работы в банке....,"[очень, порадовать, оперативность, работа, бан...","[18, 1055, 910, 29, 2, 475, 183, 3, 628, 1768,..."
3,0,Имела неосторожность оформить потреб. кредит в...,"[иметь, неосторожность, оформить, потреба, кре...","[118, 4818, 72, 2368, 9, 62, 18, 2665, 1287, 3..."
4,0,Небольшая предыстория: Нашел на сайте MDM банк...,"[небольшой, предыстория, найти, сайт, mdm, бан...","[418, 3851, 275, 87, 2, 648, 3, 4894, 2, 296, ..."
...,...,...,...,...
13994,1,"О высокой надёжности МКБ, порядочности и добро...","[высокий, надёжность, мкб, порядочность, добро...","[401, 2321, 809, 5028, 8159, 7, 3099, 126, 441..."
13995,1,"Обслуживаюсь в офисе на Чернореченской 42а, ка...","[обслуживаться, офис, чернореченский, 42а, физ...","[362, 24, 9494, 2152, 201, 14, 24, 96, 141, 66..."
13996,1,Попала сегодня в очень неприятную ситуацию. Ре...,"[попасть, сегодня, очень, неприятный, ситуация...","[517, 109, 18, 992, 53, 38, 224, 233, 3, 46, 3..."


### Выделяем данные для обучения и тестирования

In [ ]:
train, test = train_test_split(banks, test_size=0.2)

In [ ]:
train

,Score,Text,Preprocessed_texts,Sequences
idx,,,,
4535,0,Я являлась клиентом ЗАО Москомприватбанк с 201...,"[являться, клиент, зао, москомприватбанк, 2012...","[86, 13, 1752, 1045, 1038, 126, 665, 77, 21, 3..."
7266,1,"Как сотрудник другого банка (не Сити), причем ...","[сотрудник, другой, банк, сити, причём, работа...","[7, 48, 2, 1095, 314, 43, 1291, 1934, 14, 1202..."
4140,0,19.09.2015 я решила приобрести телефон в расср...,"[19.09.2015, решить, приобрести, телефон, расс...","[38, 1174, 32, 855, 265, 2151, 87, 265, 2151, ..."
6149,1,"Работал со многими банками: вклады, кредиты, п...","[работать, многий, банк, вклад, кредит, перево...","[43, 803, 2, 47, 9, 105, 262, 21, 3, 112, 85, ..."
1853,0,"Желаете потерять полдня, а потом остаться с но...","[желать, потерять, полдень, остаться, нос, заб...","[735, 610, 3324, 218, 3502, 493, 845, 158, 187..."
...,...,...,...,...
7367,0,Уже длительное время поступают звонки по повод...,"[длительный, время, поступать, звонок, повод, ...","[1380, 19, 554, 49, 386, 920, 201, 7893, 50, 1..."
212,0,Добрый день. Хочу поделиться с вами печальной ...,"[добрый, день, хотеть, поделиться, вы, печальн...","[181, 5, 39, 999, 740, 2848, 169, 29, 1803, 26..."
7807,1,Мне нравится. А хороший или не хороший? Это ка...,"[нравиться, хороший, хороший, это, водка, сего...","[590, 131, 131, 4, 109, 9989, 642, 1263, 1000,..."


In [ ]:
test

,Score,Text,Preprocessed_texts,Sequences
idx,,,,
5544,0,Мне всё труднее писать что-либо об Альфа-банке...,"[всё, трудный, писать, что-либо, альфа-банк, т...","[123, 2736, 114, 2823, 62, 112, 158, 576, 123,..."
5321,1,"Было бы не справедливо, если бы не написал это...","[справедливо, написать, отзыв, оставлять, ране...","[4884, 51, 93, 695, 478, 93, 2, 1305, 191, 320..."
2750,1,Подключился к Модулю (тогда Региональный Креди...,"[подключиться, модуль, региональный, кредит, н...","[2769, 4425, 3092, 9, 459, 717, 14, 1100, 2, 3..."
13254,0,Сегодня надо было оплатить пошлину за регистра...,"[сегодня, оплатить, пошлина, регистрация, даре...","[109, 193, 4961, 806, 2123, 122, 211, 229, 182..."
13034,0,20 января в 17 часов я обратился в единственны...,"[20, январь, 17, час, обратиться, единственный...","[315, 434, 1054, 108, 74, 443, 24, 245, 291, 4..."
...,...,...,...,...
2260,1,Являюсь вкладчиком и пользователем дебетной ка...,"[являться, вкладчик, пользователь, дебетной, к...","[86, 933, 1579, 9691, 3, 213, 434, 793, 14, 47..."
13657,0,Я являюсь вашим клиентом с июля 2007 года. В н...,"[являться, ваш, клиент, июль, 2007, год, ноябр...","[86, 42, 13, 485, 1397, 14, 459, 1149, 14, 72,..."
13106,1,Решила написать этот отзыв после некоторых поп...,"[решить, написать, отзыв, некоторый, попытка, ...","[38, 51, 93, 332, 503, 780, 242, 398, 48, 2, 2..."


### Разделяем метки классов и данные для обучения

Данные для обучения

In [ ]:
x_train_seq = train['Sequences']
y_train = train['Score']

In [ ]:
x_train_seq

idx
4535    [86, 13, 1752, 1045, 1038, 126, 665, 77, 21, 3...
7266    [7, 48, 2, 1095, 314, 43, 1291, 1934, 14, 1202...
4140    [38, 1174, 32, 855, 265, 2151, 87, 265, 2151, ...
6149    [43, 803, 2, 47, 9, 105, 262, 21, 3, 112, 85, ...
1853    [735, 610, 3324, 218, 3502, 493, 845, 158, 187...
                              ...                        
7367    [1380, 19, 554, 49, 386, 920, 201, 7893, 50, 1...
212     [181, 5, 39, 999, 740, 2848, 169, 29, 1803, 26...
7807    [590, 131, 131, 4, 109, 9989, 642, 1263, 1000,...
5886    [362, 884, 24, 2, 2995, 2227, 355, 1469, 85, 6...
759     [1930, 64, 3762, 77, 3, 6752, 2018, 3, 643, 32...
Name: Sequences, Length: 11199, dtype: object

In [ ]:
y_train

idx
4535    0
7266    1
4140    0
6149    1
1853    0
       ..
7367    0
212     0
7807    1
5886    1
759     1
Name: Score, Length: 11199, dtype: int64

Данные для тестирования

In [ ]:
x_test_seq = test['Sequences']
y_test = test['Score']

In [ ]:
x_test_seq

idx
5544     [123, 2736, 114, 2823, 62, 112, 158, 576, 123,...
5321     [4884, 51, 93, 695, 478, 93, 2, 1305, 191, 320...
2750     [2769, 4425, 3092, 9, 459, 717, 14, 1100, 2, 3...
13254    [109, 193, 4961, 806, 2123, 122, 211, 229, 182...
13034    [315, 434, 1054, 108, 74, 443, 24, 245, 291, 4...
                               ...                        
2260     [86, 933, 1579, 9691, 3, 213, 434, 793, 14, 47...
13657    [86, 42, 13, 485, 1397, 14, 459, 1149, 14, 72,...
13106    [38, 51, 93, 332, 503, 780, 242, 398, 48, 2, 2...
11685    [14, 344, 90, 119, 47, 1212, 512, 778, 189, 16...
7755     [278, 13, 2007, 86, 86, 61, 86, 236, 94, 443, ...
Name: Sequences, Length: 2800, dtype: object

In [ ]:
y_test

idx
5544     0
5321     1
2750     1
13254    0
13034    0
        ..
2260     1
13657    0
13106    1
11685    0
7755     0
Name: Score, Length: 2800, dtype: int64

## Создаем мешок слов

In [ ]:
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        for index in sequence:
            results[i, index] += 1.
    return results

In [ ]:
x_train = vectorize_sequences(x_train_seq, max_words)

In [ ]:
x_test = vectorize_sequences(x_test_seq, max_words)

In [ ]:
x_train[0][:100]

array([0., 0., 7., 8., 2., 2., 1., 3., 0., 0., 0., 1., 0., 4., 0., 0., 1.,
       2., 2., 0., 0., 5., 0., 0., 2., 3., 0., 1., 0., 2., 0., 0., 0., 0.,
       0., 0., 0., 1., 1., 1., 2., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 1., 0., 0., 0., 2., 0., 0., 0., 2., 0., 0., 0., 0., 1., 3., 0.,
       0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 2., 0.,
       1., 1., 0., 0., 0., 0., 0., 0., 1., 2., 2., 0., 1., 0., 0.])

In [ ]:
len(x_train[0])

10000

## Создаем модель машинного обучения

In [ ]:
lr = LogisticRegression(random_state=random_state, max_iter=500)

## Обучаем модель машинного обучения

In [ ]:
lr.fit(x_train, y_train)

LogisticRegression(max_iter=500, random_state=42)

## Оцениваем качество обучения на тестовом наборе данных

Определяем долю правильных ответов (accuracy) на тестовом наборе данных

In [ ]:
lr.score(x_test, y_test)

0.9492857142857143

## Применяем модель для определения тональности отзыва на банк

**Позитивный отзыв**

In [ ]:
positive_text = """Брал кредит в Мегабанке на автомобиль. Выдали за один день. Никаких скрытых комиссий и переплат. 
У банка удобное мобильное приложение, через которое можно быстро отправить ежемесячный платеж. 
Досрочное гасить начал через три месяца. Я доволен оперативностью и удобством. Огромное спасибо!
"""

Подготовка текста к обработке

In [ ]:
positive_preprocessed_text = preprocess(positive_text, stop_words, punctuation_marks, morph)

In [ ]:
positive_preprocessed_text

['брать',
 'кредит',
 'мегабанк',
 'автомобиль',
 'выдать',
 'день',
 'никакой',
 'скрытый',
 'комиссия',
 'переплата',
 'банк',
 'удобный',
 'мобильный',
 'приложение',
 'который',
 'быстро',
 'отправить',
 'ежемесячный',
 'платёж',
 'досрочный',
 'гасить',
 'начать',
 'месяц',
 'довольный',
 'оперативность',
 'удобство',
 'огромный',
 'спасибо']

In [ ]:
positive_seq = text_to_sequence(positive_preprocessed_text, word_to_index)

In [ ]:
positive_seq

[170,
 9,
 963,
 136,
 5,
 81,
 1890,
 80,
 1028,
 2,
 287,
 305,
 763,
 8,
 128,
 205,
 489,
 30,
 358,
 1422,
 278,
 34,
 299,
 910,
 1316,
 312,
 75]

In [ ]:
positive_bow = vectorize_sequences([positive_seq], max_words)

In [ ]:
positive_bow[0][0:100]

array([0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

Выполняем распознавание

In [ ]:
result = lr.predict(positive_bow)

In [ ]:
result

array([1])

**Негативный отзыв**

In [ ]:
negative_text = """Взял кредит в ТакСебеБанке на автомобиль. В договор включили обязательный контракт
на помощь на дороге, который мне не нужен. Узнал об этом только во время подписания договора, иначе бы отказался.
Альтернативы была страхование жизни, но мне это даже не предложили. Скорее всего, менеджер продвигает
продажи услуг этой компании в ущерб интересов клиента. Как минимум, непорядочно и непрофессионально.
У банка ужасное мобильное приложение, из-за которого с меня взяли штраф 10 тыс.руб. По требованиям 
банка после покупки автомобиля в приложении нужно загрузить ПТС. Я загрузил и проверил, что ПТС в приложении есть.
Но через некоторое время ПТС из приложения пропал и с меня взяли штраф. Никому не рекомендую связываться с ТакСебеБанком.
"""

In [ ]:
negative_preprocessed_text = preprocess(negative_text, stop_words, punctuation_marks, morph)
negative_seq = text_to_sequence(negative_preprocessed_text, word_to_index)
negative_bow = vectorize_sequences([negative_seq], max_words)

In [ ]:
result = lr.predict(negative_bow)

In [ ]:
result

array([0])

In [ ]:
result = lr.predict_proba(negative_bow)

In [ ]:
result

array([[0.99854494, 0.00145506]])